In [1]:
%matplotlib inline

import pymongo
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tqdm import tqdm
import talib
import gc
from typing import Union


In [2]:
class Chast:

    def __init__(self, client, db_name='Fields'):
        self.client = client
        self.db = client[db_name]
        self._config()
        self.data = dict()
        self.results = dict()
        self.initial_capital = 100000000
        
    def _config(self):
        # 設定繪圖、TQDM讀取條、小數點位數、顯示視窗長度
        pd.options.plotting.backend = "matplotlib"
        tqdm.pandas(desc="progress-bar")
        plt.rcParams['font.family'] = ['Microsoft JhengHei'] # 中文標籤
        plt.rcParams['axes.unicode_minus'] = False # 負號
        plt.rcParams['figure.figsize'] = (16,12)
        plt.rcParams.update({'font.size': 12})
        pd.set_option('display.max_rows', 200)
        pd.set_option('display.float_format', lambda x: '%.3f' % x)
        pd.options.display.float_format = '{:,.4f}'.format

    def _result(self):
        pf = self.results['PROFIT']
        com = self.results['COMMISION']
        self.results['RESULT'] = (pf - com).sum(axis=1).cumsum()

    def _maxdrawdown(self):
        se = self.results['RESULT']
        mdd = 0
        mdd_ = list()
        h = 0
        for i in se:
            if i > h:
                h = i
            mdd = i - h
            mdd_.append(mdd)
        self.results['MDD'] = pd.Series(mdd_, index=se.index, name='MDD')

    def _sharpe_ratio(self, roll: int = 120):
        """
        計算Sharpe ratio
        預設roll 120天=半年
        roll
        """
        pf = self.results['RESULT'] 
        self.results['SHARPE RATIO'] = (pf-pf.shift(1)).rolling(roll).apply(lambda x: x.mean() / x.std() if x.std() != 0 else 0, raw=True)
    
    def _holding(self):

        bs = self.results['BS']
        if '收盤價' in self.data.keys():
            holding = bs.fillna(0) * self.data['收盤價'].loc[bs.index, bs.columns] * 1000
        else:
            self.get_from_mongo('收盤價')
            holding = bs.fillna(0) * self.data['收盤價'].loc[bs.index, bs.columns] * 1000
        self.results['HOLDING'] = holding

    def _capital(self):
        self.results['CAPITAL NET'] = self.results['CAPITAL'][:-1]

    def set_result(self):
        self._result()
        self._maxdrawdown()
        self._holding()
        self._sharpe_ratio(120)
        self._capital()

    def get_chart(self):
        keys = self.results
        if 'RESULT' not in keys:
            self.set_result()
        keys = self.results
        fig ,ax = plt.subplots(8, 1, figsize=(16, 16), gridspec_kw={'height_ratios': [2, 1, 1, 1, 1, 1, 2, 1]})
        
        if 'RESULT' in keys:
            ax[0].plot(self.results['RESULT'][:-1])
            ax[0].set_title(f'累計損益')
            ax[0].grid(True)
            ax[0].yaxis.set_major_formatter('{x:,.0f}')
            ax[0].xaxis.set_minor_locator(mdates.YearLocator())
        if 'MDD' in keys:
            ax[1].plot(self.results['MDD'][:-1], color='orange')
            ax[1].grid(True)
            ax[1].set_title('MaxDrawdown')
            ax[1].yaxis.set_major_formatter('{x:,.0f}')
        if 'BS' in keys:
            ax[2].plot(self.results['BS'].fillna(0).astype(bool).sum(axis=1)[:-1], color='green')
            ax[2].set_title('累計股數')
        if 'HOLDING' in keys:
            ax[3].plot(self.results['HOLDING'].sum(axis=1)[:-1], color='lightgreen')
            ax[3].set_title('累計市值')
            ax[3].yaxis.set_major_formatter('{x:,.0f}')
        if 'SHARPE RATIO' in keys:
            ax[4].plot(self.results['SHARPE RATIO'][:-1], color='lightblue')
            ax[4].grid(True)
            ax[4].set_title('Rolling Sharpe Ratio')
            ax[4].yaxis.set_major_formatter('{x:.3f}')
        if 'CAPITAL' in keys:
            ax[5].plot(self.results['CAPITAL NET'][:-1], color='darkgreen')
            ax[5].grid(True)
            ax[5].set_title('Capital')
            ax[5].yaxis.set_major_formatter('{x:,.0f}')
        if 'HEDGE' in keys:
            ax[6].plot(self.results['RESULT'] + self.results['HEDGE'].cumsum())
            ax[6].grid(True)
            ax[6].set_title('避險後損益')
            ax[6].yaxis.set_major_formatter('{x:,.0f}')
            ax[7].plot(self.results['HEDGE'].cumsum())
            ax[7].grid(True)
            ax[7].set_title('避險單獨損益')
            ax[7].yaxis.set_major_formatter('{x:,.0f}')

        fig.tight_layout()
        plt.show()        

    def get_from_mongo(self, elements, nodate: bool=False,
     start: Union[datetime.datetime, None]=None, 
     end: Union[datetime.datetime, None]=None):
        """
        elements: Element, 表示要在Mongo中抓哪些資料, 可用list包起來好幾項
        start: 開始時間
        end: 結束時間
        """
        if type(elements) != list:
            if elements not in self.data:
                if nodate:
                    self.data[elements] = pd.DataFrame(self.db[elements].find({}, {'_id': 0}))
                else:
                    self.data[elements] = pd.DataFrame(self.db[elements].find({"日期": {'$gt': start, '$lt': end}}, {'_id': 0})).set_index('日期')
        else:
            for e in elements:
                if e not in self.data:
                    if nodate:
                        self.data[e] = pd.DataFrame(self.db[e].find({}, {'_id': 0}))
                    else:
                        self.data[e] = pd.DataFrame(self.db[e].find({"日期": {'$gt': start, '$lt': end}}, {'_id': 0})).set_index('日期')

    def set_data(self, name: str, df: pd.DataFrame):
        """
        name: 指定data名稱
        data: 放入data
        """
        self.data[name] = df
    
    def set_result(self, pf: pd.DataFrame, bs: pd.DataFrame, com: pd.DataFrame, ep: pd.DataFrame, ed: pd.DataFrame, ca: pd.Series, hedge: pd.Series=None, hbs: pd.Series=None):
        """
        pf: 損益明細
        bs: 部位明細
        com: 費用明細
        ep: 進場價格明細
        ed: 進場時間明細
        ca: 剩餘資金變化
        hedge: 避險損益
        """
        self.results['PROFIT'] = pf
        self.results['BS'] = bs
        self.results['COMMISION'] = com
        self.results['ENTRY PRICE'] = ep
        self.results['ENTRY DATE'] = ed
        self.results['CAPITAL'] = ca
        if hedge != None:
            self.results['HEDGE'] = hedge
        if hbs != None:
            self.results['HBS'] = hbs
        
    def creat_to_daily(self, df: pd.DataFrame):
        n = pd.DataFrame(columns=self.data['還原收盤價'].columns, index=self.data['還原收盤價'].index.union(df.index))
        for i in df.index:
            for c in df.columns:
                n.at[i, str(c)] = df.at[i, c]
        return n.fillna(method='ffill')

In [3]:
from IPython.display import display
client = pymongo.MongoClient()
chast = Chast(client, 'Fields')
index = Chast(client, 'Index')

### <font color=osheet.range> 策略步驟 </font>
* 1. 下載需要的資料
* 2. 設定要用到的資料，時間點要對齊收盤
* 3.

##### 1. 下載需要的資料

In [4]:
getting_list = [
    '還原開盤價', '還原最高價', '還原最低價', '還原收盤價', '收盤價', '成交金額(千)', '總市值(億)', '漲幅(%)'
]
start = datetime.datetime(2016, 1, 1)
# end = datetime.datetime(2022, 10, 11)
end = datetime.datetime.today()
chast.get_from_mongo(getting_list, start=start, end=end)

index_list = [
    'TWA00', 'VIX'
]
index.get_from_mongo(index_list, start=start, end=end)
index.get_from_mongo('產業名稱', nodate=True)


##### 2. 設定要用到的資料

##### 將各產業分開來，並且計算每天的漲跌

In [5]:
index.data['產業名稱']['股票代號'] = index.data['產業名稱']['股票代號'].astype(str)
close = {}
value = {}

for i in index.data['產業名稱']['產業名稱'].unique():
    ticker = index.data['產業名稱'][index.data['產業名稱']['產業名稱'] == i]['股票代號']
    close[i] = chast.data['還原收盤價'].loc[:, ticker]
    se1 = chast.data['總市值(億)'].loc[:, ticker].sum(axis=1)
    se1.name = '總市值(億)'
    se2 = chast.data['成交金額(千)'].loc[:, ticker].sum(axis=1)
    se2.name = '成交金額(千)'
    value[i] = pd.concat([se1, se2], axis=1)



##### 現在我擁有各產業的市值累計值，直接以這個資料來測試簡單突破

In [6]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt

class Strategy(bt.Strategy):
    params = dict(
        stdlen=24,
        malen=24,
    )
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        std = [bt.ind.StdDev(d, period=self.p.stdlen) for d in self.datas]
        ma = [bt.ind.MovingAverageSimple(d, period=self.p.malen) for d in self.datas]
        self.crossover = {d: d.close > (m+s) for d, m, s in zip(self.datas, ma, std)}
        self.crossunder = {d: d.close < (m-s) for d, m, s in zip(self.datas, ma, std)}
        self.order = None
        self.buyprice = None
        self.buycomm = None

    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                    
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            else:
                self.log(
                    'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def nodify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                (trade.pnl, trade.pnlcomm))


    def next(self):
        # self.log('Close,%s %.2f' % (self.data._name, self.dataclose[0]))

        if self.order:
            return
        crossover = {d for d, v in self.crossover.items() if v}
        crossunder = {d for d, v in self.crossunder.items() if v}

        posdata = [d for d, pos in self.getpositions().items() if pos]

        for d in (d for d in crossover if d not in posdata):
            self.log('BUY CREATE, %s %.2f' % (d._name, d.close[0]))
            self.order = self.buy(d, size=int(self.broker.get_cash()/d.close/50))

        for d in (d for d in posdata if d in crossunder):
            self.log('SELL CREATE, %s %.2f' % (d._name, d.close[0]))
            self.order = self.sell(d)

class feed_data(bt.feeds.PandasData):
    params = (
        ('close', 0),
        ('volume', 1),
        ('open', 0),
        ('high', -1),
        ('low', -1),
        ('openinterest', -1)
    )

if __name__ == '__main__':
    cerebro = bt.Cerebro(stdstats=True)
    cerebro.addstrategy(Strategy)
    
    client = pymongo.MongoClient()
    db = client['Fields']

    # Create a Data Feed
    for k, v in value.items():
        data = feed_data(dataname=v, name=k)
        # Add the Data Feed to Cerebro
        cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000000.0)
    cerebro.broker.setcommission(commission=0.002)
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()
    #cerebro.plot(iplot=False)
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000000.00
2016-02-03, BUY CREATE, 電子–通信網路 18118.00
2016-02-03, BUY CREATE, 電子–電腦及週邊設備 16073.10
2016-02-03, BUY CREATE, 貿易百貨 5083.00
2016-02-03, BUY CREATE, 鋼鐵工業 4775.70
2016-02-03, BUY CREATE, 電子–半導體 62456.50
2016-02-03, BUY CREATE, 電子商務 493.40
2016-02-03, BUY CREATE, 橡膠工業 3024.10
2016-02-03, BUY CREATE, 電子–電子通路 2115.10
2016-02-03, BUY CREATE, 文化創意 906.30
2016-02-03, BUY CREATE, 紡織纖維 5266.10
2016-02-03, BUY CREATE, 塑膠工業 14981.70
2016-02-03, BUY CREATE, 電器電纜 648.30
2016-02-15, BUY EXECUTED, Price: 18255.50, Cost: 2008105.00, Comm 4016.21
2016-02-15, BUY EXECUTED, Price: 16105.40, Cost: 1997069.60, Comm 3994.14
2016-02-15, BUY EXECUTED, Price: 5091.90, Cost: 2001116.70, Comm 4002.23
2016-02-15, BUY EXECUTED, Price: 4878.00, Cost: 2039004.00, Comm 4078.01
2016-02-15, BUY EXECUTED, Price: 62461.70, Cost: 1998774.40, Comm 3997.55
2016-02-15, BUY EXECUTED, Price: 489.80, Cost: 1985159.40, Comm 3970.32
2016-02-15, BUY EXECUTED, Price: 3030.20, Cost: 2002962.20, Co

In [9]:
for d in cerebro.datas:
    if d._name != '水泥工業':
        d.plotinfo.plot = False

In [10]:
cerebro.plot(iplot=False)

[[<Figure size 1152x864 with 5 Axes>]]